In [26]:
print("all ok")

all ok


In [ ]:
!pip install huggingface_hub
# !pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

  Using cached numpy-1.23.4-cp310-cp310-win_amd64.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\ASIF\\AppData\\Local\\Temp\\pip-unpack-rt27pqil\\numpy-1.23.4-cp310-cp310-win_amd64.whl'
Consider using the `--user` option or check the permissions.



In [ ]:
!pip install llama-cpp-python==0.1.78

  Using cached llama_cpp_python-0.1.78.tar.gz (1.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Failed to build llama-cpp-python


  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [505 lines of output]
      
      
      --------------------------------------------------------------------------------
      -- Trying 'Ninja (Visual Studio 17 2022 x64 v144)' generator
      --------------------------------
      ---------------------------
      ----------------------
      -----------------
      ------------
      -------
      --
      Not searching for unused variables given on the command line.
      CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
        Compatibility with CMake < 3.10 will be removed from a future version of
        CMake.
      
        Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
        to tell CMake that the project requires at least <min> but has been updated
        to work with policies introduced by <max> or earlier.
      
      
   

# pip install -qU langchain-pinecone

In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone, pinecone
import pinecone
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

In [2]:
from langchain_pinecone import PineconeVectorStore

e:\DataScience\anaconda\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the data

In [3]:
def load_data(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    
    docs=loader.load()
    
    return docs

In [ ]:
pip install pypdf


Note: you may need to restart the kernel to use updated packages.


In [4]:
extracted_data=load_data("data/")

### Creating Text chunks

In [5]:
def creating_chunks(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
    text_chunks=text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [6]:
text_chunks=creating_chunks(extracted_data)

In [7]:
len(text_chunks)

7080

### Downloading OpenAI Embeddings

In [ ]:
import os

os.environ["OPENAI_API_KEY"]=""

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()

embeddings=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])

In [10]:
embed=embeddings.embed_query("How are you?")

In [11]:
len(embed)

1536

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Set env variable for LangChain
os.environ["PINECONE_API_KEY"] = ""

# Init Pinecone v4
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Set env variable for LangChain
os.environ["PINECONE_API_KEY"] = ""

# Init Pinecone v4
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

if "llama2" not in pc.list_indexes().names():
    pc.create_index(
        name="llama2",
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Create vector store from texts
batch_size = 50  # adjust based on your rate limits
for i in range(0, len(text_chunks), batch_size):
    batch = text_chunks[i:i+batch_size]
    PineconeVectorStore.from_texts(
        [t.page_content for t in batch],
        embeddings,
        index_name="llama2"
    )


In [ ]:
index_name="llama2"
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [14]:
query="what is allergy?"

result=docsearch.similarity_search(query=query, k=3)



In [15]:
result[0].page_content

'Allergen —A foreign substance that provokes an\nimmune reaction in some sensitive people but not\nin most others.\nAnaphylaxis —An exaggerated, life-threatening\nhypersensitivity reaction to a previously encoun-\ntered antigen.\nAntibody —An immune system protein made to\nfight infection.\nAntigen —A foreign substance detected that pro-\nvokes an immune reaction.\nAbsence of any reaction to injected areas indi-\ncates lack of T cell responsiveness, a condition called'

In [16]:
prompt_template="""
Use the following information as context to answer the user question.

context : {context}
Question : {question}

only returns the helpful answer below and nothing else.

Helpful Answer : 

"""

In [19]:
PROMPT=PromptTemplate(
    input_variables=["context","question"],
    template=prompt_template
)

chain_type_kwargs={"prompt": PROMPT}

In [20]:
from langchain_community.llms import CTransformers

In [21]:
llm = CTransformers(
    model="E:/GEN AI/End2End-Medical-Chatbot-using-Llama2/models/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    max_new_tokens=512,
    temperature=0.5
)

In [22]:
qa_model = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input=input(f"Write your query : ")
    
    if user_input is not None:
        result=qa_model({'query':user_input})
        print("Response : ", result["result"])
        
    elif user_input=="exit":
        print("existing")
        break
    else:
        continue
    
    

In [23]:
user_input=input(f"Write your query : ")

In [24]:
user_input

'what is acne'

In [27]:
result=qa_model({'query':user_input})
print("Response : ", result["result"])

Response :  Acne is a common skin condition that results from com-
bined exposure to an allergen (an allergy-causing sub-
stance) and the skin's immune response to it, leading to
inflammation of the skin's oil glands.


In [26]:
result = qa_model.invoke({'query': user_input})
print("Response:", result["result"])


Response: Acne results from a combination of hormonal changes, excess oil production, and clogged pores in the skin.
